In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('train.csv')

In [2]:
data= np.array(data)

In [3]:
len(data)

42000

In [4]:
#Train test split 80:20
test = data[int(len(data)*0.8):]

train = data[:int(len(data)*0.8)]

In [5]:
len(train)

33600

In [6]:
len(test)

8400

In [7]:
len(train[0]) # 1 label col, 784 pixel values

785

In [8]:
digit_labels = train[:,0] #label col separated out
len(digit_labels)

33600

In [9]:
# TEMPORARILY MADE INTO 42 instances to not f up PC
digit_pixels = train[:42,1:] #training data, 42000 instances of 28x28 pixel values ,then scaling to 0-1
len(digit_pixels)

42

In [10]:

weights = np.random.uniform(-0.0001,0.0001,size=(10,784))

len(weights[0])

784

In [11]:
bias = np.random.uniform(-0.1,0.1,10)

bias

array([ 0.01450097,  0.00174535,  0.02366215,  0.01321934,  0.05561376,
        0.01677179, -0.06680002,  0.01238687, -0.05452103, -0.05316519])

In [12]:
def sigmoid(x): #sigmoid func to squish all inputs into range 0 to 1
    return 1 / (1 + np.exp(-x))

In [13]:
weights @ digit_pixels[3] + bias

array([-0.06893485, -0.11166313, -0.13881483,  0.09741783,  0.08142572,
       -0.09993679, -0.02442895,  0.04445895,  0.02671694, -0.07368261])

In [14]:
output=sigmoid(weights @ digit_pixels[41] + bias) # @ is matrix multiplication 
output

array([0.46826437, 0.48921233, 0.53974376, 0.54503692, 0.48795143,
       0.50446737, 0.49597242, 0.52434299, 0.51266741, 0.50720831])

In [15]:
x=0 
one_hot = np.zeros(10)
one_hot[digit_labels[x]]=1 #One hot encoder e.g. [0,0,0,1,0,0,0,0,0] for calculating cost func 
print(one_hot, "digit =", digit_labels[x])

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] digit = 1


In [16]:
cost = (one_hot - output)**2 #Calculating cost function
cost

array([0.21927152, 0.26090404, 0.29132333, 0.29706525, 0.2380966 ,
       0.25448733, 0.24598864, 0.27493557, 0.26282788, 0.25726027])

In [17]:
bias

array([ 0.01450097,  0.00174535,  0.02366215,  0.01321934,  0.05561376,
        0.01677179, -0.06680002,  0.01238687, -0.05452103, -0.05316519])

In [18]:
# The MAIN block
learning_rate = -0.1

for t in range(1):
    digit_pixels = ((train[t*42:42*(t+1),1:] - np.min(train[t*42:42*(t+1),1:]))/(np.max(train[t*42:42*(t+1),1:])-np.min(train[t*42:42*(t+1),1:]))) #Scaling to 0-1

    for _ in range(50): #training on repeat
        dCdw = np.zeros(shape=(10,784))
        dCdb = np.zeros(10)
        cost = 0
        
        for i in range (len(digit_pixels)): #Now just iterate through all training examples
        
            #Forward Propagation
            z = weights @ digit_pixels[i] + bias
            output = sigmoid(z)
            
            one_hot = np.zeros(10)
            one_hot[digit_labels[i]]=1
        
            cost = cost + np.sum((one_hot - output)**2)
            
            #Backpropagation calculus
        
            dC_da = -2*(one_hot - output)
            dadz = (np.exp(-z) / (1 + np.exp(-z))**2)
            
            for x in range (len(weights[0])): #iterating through weights column by column
                # weights              
                dzdw = digit_pixels[i,x] #This one input, affects a whole column of weights
                dC_dw = dC_da * dadz * dzdw
                weights[:,x] += learning_rate*dC_dw/len(digit_pixels)
     
            #Biases
            dzdb = 1
            dC_db = dC_da * dadz * dzdb
            bias += learning_rate*dC_db/len(digit_pixels)
        
        #Updating weights
        #weights += dCdw
        #bias += dCdb
            


In [19]:
cost

13.39906250571421

In [20]:
test_label = test[:,0]
test_pixels = (test[:,1:] -np.min(test[:,1:]))/(np.max(test[:,1:])-np.min(test[:,1:]))

In [21]:
correct_ans = []
#check accuracy on test set
correct = 0
for i in range(len(test)):
    z = weights @ test_pixels[i] + bias
    output = sigmoid(z)

    if test_label[i] == np.argmax(output):
        correct += 1
        correct_ans.append([test_label[i],i])
        
        
print(correct/len(test))

        


0.5341666666666667


In [22]:
#Check accuracy on training set
correct = 0
for i in range(42):
    z = weights @ digit_pixels[i] + bias
    output = sigmoid(z)

    if digit_labels[i] == np.argmax(output):
        correct += 1
        
print(correct/42)

1.0


In [23]:
#random guess and check (10%)
correct = 0
for i in range(len(test)):

    if test_label[i] == np.random.randint(0,10):
        correct += 1
        
print(correct/len(test))


0.10535714285714286


In [24]:
#Check how it actually looks
ascii_chars = "@%#*+=-:. "
scale = len(ascii_chars) - 1

pixels = test_pixels[164]  # 784-length flattened image

for i in range(28):
    row = pixels[i * 28:(i + 1) * 28]
    line = ''.join([ascii_chars[min(int(p * scale), scale)] for p in row])
    print(line)


@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@%+++%@@@@@@@@@@@@
@@@@@@@@*=.-...=*@@@@@@@@@@@
@@@@@@*...........+@@@@@@@@@
@@@@@%.............:%@@@@@@@
@@@@@+....======-...:%@@@@@@
@@@@@#..:%@@@@@@@%:..*@@@@@@
@@@@@@*=*@@@@@@@@#-..*@@@@@@
@@@@@@@@@@@@@@@@@:..-@@@@@@@
@@@@@@@@@@@@@@@%:...*@@@@@@@
@@@@@@@@@@@@@@=:...-@@@@@@@@
@@@@@@@@@@@@@=...:-@@@@@@@@@
@@@@@@@@@@*-:...:@@@@@@@@@@@
@@@@@@%*-:.......-*@@@@@@@@@
@@@@@-.............:*@@@@@@@
@@@@@ .....::::......:*@@@@@
@@@@@*=..**@@@@@*+.....*@@@@
@@@@@@@@@@@@@@@@@@@+....#@@@
@@@@@@@@@@@@@@@@@@@@#....@@@
@@@@@@@@@@@@@@@@@@@@@#...@@@
@@@@@@@@@@@@@@@@@@@@@@+++@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [25]:
from PIL import Image

img = Image.open("number.png") #Replace your_image.jpg with the path to your image

In [26]:
img

In [27]:
img_array = np.array(img)

In [28]:
img_array = (img_array[:,:,0]).flatten()

In [29]:
#Check how it actually looks
ascii_chars = "@%#*+=-:. "
scale = len(ascii_chars) - 1

pixels = img_array  # 784-length flattened image

for i in range(28):
    row = pixels[i * 28:(i + 1) * 28]
    line = ''.join([ascii_chars[min(int(p * scale), scale)] for p in row])
    print(line)


@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@       @@@@@@@@@@@@@@@
@@@@@            @@@@@@@@@@@
@@@@             @@@@@@@@@@@
@@@      @        @@@@@@@@@@
@@@     @@@@@     @@@@@@@@@@
@@@    @@@@@@@    @@@@@@@@@@
@@@   @@@@@@@@    @@@@@@@@@@
@@@   @@@@@@@     @@@@@@@@@@
@@@    @@@@@      @@@@@@@@@@
@@@      @@      @@@@@@@@@@@
@@@             @@@@@@@@@@@@
@@@@            @@@@@@@@@@@@
@@@@@@             @@@@@@@@@
@@@@@@@@             @@@@@@@
@@@@@@@@              @@@@@@
@@@@@@@     @@@@      @@@@@@
@@@@@@@    @@@@@@@     @@@@@
@@@@@@@    @@@@@@@@    @@@@@
@@@@@@@    @@@@@@@@    @@@@@
@@@@@@@     @@@@@@@    @@@@@
@@@@@@@       @@@      @@@@@
@@@@@@@@               @@@@@
@@@@@@@@@             @@@@@@
@@@@@@@@@@@         @@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [30]:
z = weights @ test_pixels[164] + bias
output = sigmoid(z)

In [31]:
print(output, output.argmax())

[0.08237401 0.02622382 0.21341344 0.06929656 0.05372317 0.03281192
 0.06243028 0.12901288 0.06283475 0.09943481] 2
